In [1]:
import modelseedpy

In [2]:
import cobrakbase
kbase = cobrakbase.KBaseAPI()

cobrakbase 0.3.1


In [3]:
template = kbase.get_from_ws('GramNegModelTemplateV3', 'NewKBaseModelTemplates')

In [4]:
from modelseedpy import MSBuilder

In [8]:
from cobra.core import Model
model = Model('test')

In [9]:
biomass = template.biomasses[0]

In [10]:
reaction = biomass.build_biomass(model, '0', True)

In [11]:
for m in reaction.metabolites:
    print(m, m.notes)

cpd00010_c0 {'modelseed_template_id': 'cpd00010_c'}
cpd11493_c0 {'modelseed_template_id': 'cpd11493_c'}
cpd12370_c0 {'modelseed_template_id': 'cpd12370_c'}
cpd00003_c0 {'modelseed_template_id': 'cpd00003_c'}
cpd00006_c0 {'modelseed_template_id': 'cpd00006_c'}
cpd00205_c0 {'modelseed_template_id': 'cpd00205_c'}
cpd00254_c0 {'modelseed_template_id': 'cpd00254_c'}
cpd10516_c0 {'modelseed_template_id': 'cpd10516_c'}
cpd00063_c0 {'modelseed_template_id': 'cpd00063_c'}
cpd00009_c0 {'modelseed_template_id': 'cpd00009_c'}
cpd00099_c0 {'modelseed_template_id': 'cpd00099_c'}
cpd00149_c0 {'modelseed_template_id': 'cpd00149_c'}
cpd00058_c0 {'modelseed_template_id': 'cpd00058_c'}
cpd00015_c0 {'modelseed_template_id': 'cpd00015_c'}
cpd10515_c0 {'modelseed_template_id': 'cpd10515_c'}
cpd00030_c0 {'modelseed_template_id': 'cpd00030_c'}
cpd00048_c0 {'modelseed_template_id': 'cpd00048_c'}
cpd00034_c0 {'modelseed_template_id': 'cpd00034_c'}
cpd00016_c0 {'modelseed_template_id': 'cpd00016_c'}
cpd00220_c0 

In [42]:
template = MSTemplateBuilder.from_dict(get_template('template_gram_neg')).build()

In [62]:
model = kbase.get_from_ws('GCF_000009725.1.mdl', 56739)
template = kbase.get_from_ws(model.template_ref)

In [91]:
template.biomasses[0].keys()

dict_keys(['cellwall', 'cofactor', 'dna', 'energy', 'id', 'lipid', 'name', 'other', 'protein', 'rna', 'templateBiomassComponents', 'type'])

In [221]:
class TemplateComponent:
    
    def __init__(self, component_class, coefficient, coefficient_type, compound, linked_compounds):
        self.component_class = component_class
        self.coefficient = coefficient
        self.coefficient_type = coefficient_type
        self.compound = compound
        self.linked_compounds = linked_compounds
        
    def __str__(self):
        return f'{self.compound.id} [{self.component_class}:{self.coefficient_type}]'

biomass_data = template.biomasses[0]
biomass_components = {}
for o in biomass_data['templateBiomassComponents']:
    c_cpd = template.compcompounds.get_by_id(o['templatecompcompound_ref'].split('/')[-1])
    linked_cpds = {}
    for i in range(len(o['linked_compound_refs'])):
        l_cpd = template.compcompounds.get_by_id(o['linked_compound_refs'][i].split('/')[-1])
        value = o['link_coefficients'][i]
        linked_cpds[l_cpd] = value
    template_component = TemplateComponent(o['class'], o['coefficient'], o['coefficient_type'], 
                                           c_cpd, linked_cpds)
    if template_component.component_class not in biomass_components:
        biomass_components[template_component.component_class] = {
            'coefficient': biomass_data[template_component.component_class],
            'components': []
        }
    biomass_components[template_component.component_class]['components'].append(template_component)

In [227]:
biomass_components_class = {}
classMW = {}
classMol = {}
classMolFraction = {}
classMolSplitCount = {}
classMolSplitFraction = {}
classMolSplitMW = {}

classHash = {}
includedComps = []
for c_type in biomass_components:
    v = biomass_components[c_type]['coefficient']
    classHash[c.component_class] = v
    for c in biomass_components[c_type]['components']:
        includedComps.append(c)
        #print(c.coefficient_type)
        if c.coefficient_type == 'MOLFRACTION':
            if c.component_class not in classMolFraction:
                classMolFraction[c.component_class] = 0
            classMolFraction[c.component_class] += -1 * c.coefficient
            if c.compound.compound.mass and c.compound.compound.mass > 0:
                if c.component_class not in classMW:
                    classMW[c.component_class] = 0
                classMW[c.component_class] += -1 * c.compound.compound.mass * c.coefficient;
        elif c.coefficient_type == 'MOLSPLIT':
            if c_type not in classMolSplitCount:
                classMolSplitCount[c_type] = 0
            classMolSplitCount[c_type] += 1
            if c.compound.compound.mass and c.compound.compound.mass > 0:
                if c_type not in classMolSplitMW:
                    classMolSplitMW[c_type] = 0
                classMolSplitMW[c_type] += c.compound.compound.mass

In [228]:
classMassFraction = {}
classMassSplitCount = {}
classMassSplitMol = {}
classMassSplitFraction = {}
for clazz in classHash:
    if clazz not in classMassFraction:
        classMassFraction[clazz] = 0
    if clazz not in classMolFraction:
        classMolFraction[clazz] = 0
    if clazz not in classMW:
        classMW[clazz] = 0
    if clazz not in classMol:
        classMol[clazz] = 0
    if clazz not in classMolSplitCount:
        classMolSplitCount[clazz] = 0
        classMolSplitMW[clazz] = 0
    if clazz not in classMassSplitCount:
        classMassSplitCount[clazz] = 0
        classMassSplitMol[clazz] = 0
    totalSplt = classMolSplitCount[clazz] + classMassSplitCount[clazz]
    mass = (1 - classMassFraction[clazz]) * classHash[clazz] #?
    if mass > 0:
        remainingMolFraction = (1 - classMolFraction[clazz])
        if totalSplt > 0:
            massSplitMolFraction = remainingMolFraction * classMassSplitCount[clazz] / totalSplt
            molSplitMolFraction = remainingMolFraction * classMolSplitCount[clazz] / totalSplt
            classMW[clazz] += molSplitMolFraction * classMolSplitMW[clazz] / classMolSplitCount[clazz]
            if classMassSplitCount[clazz] > 0:
                classMW[clazz] += massSplitMolFraction * classHash[clazz] / (classMassSplitMol[clazz] / classMassSplitCount[clazz])
            classMassSplitFraction[clazz] = massSplitMolFraction;
            classMolSplitFraction[clazz] = molSplitMolFraction;
            if classMW[clazz] > 0:
                classMol[clazz] = mass / classMW[clazz]
            else:
                classMol[clazz] = 1

In [229]:
compoundHash = {}
for c in includedComps:
    coef = c.coefficient
    clazz = c.component_class
    z_coef = 0
    if c.coefficient_type == 'MOLFRACTION':
        z_coef = coef * classMol[clazz] * 1000;
    elif c.coefficient_type == 'MOLSPLIT':
        z_coef = coef * classMol[clazz] * classMolSplitFraction[clazz] * 1000 / classMolSplitCount[clazz]
        error = z_coef - bio1[c.compound.id + '0']
        #print(c.compound.id, c.compound.name, c.component_class,  z_coef, bio1[c.compound.id + '0'])
    elif c.coefficient_type == 'MULTIPLIER':
        z_coef = coef * classHash[clazz]
    elif c.coefficient_type == 'EXACT':
        z_coef = coef
    if c.compound.id not in compoundHash:
        compoundHash[c.compound.id] = 0
    compoundHash[c.compound.id] += z_coef
for k in compoundHash:
    print(k, template.compcompounds.get_by_id(k).name, compoundHash[k], bio1[k + '0'])

cpd00002_c ATP 0.0 -41.2913947104178
cpd00038_c GTP 0.0 -0.0424026391792249
cpd00052_c CTP 0.0 -0.0261242266150642
cpd00062_c UTP 0.0 -0.0282246669459237
cpd00115_c dATP 0 -0.0141357803930648
cpd00241_c dGTP 0 -0.012738326818085
cpd00356_c dCTP 0 -0.012738326818085
cpd00357_c TTP 0 -0.0141357803930648
cpd00023_c L-Glutamate 0.0 -0.261005044219309
cpd00033_c Glycine 0.0 -0.409331301687739
cpd00035_c L-Alanine 0.0 -0.267560900902997
cpd00039_c L-Lysine 0.0 -0.323695423757071
cpd00041_c L-Aspartate 0.0 -0.14832625746843
cpd00051_c L-Arginine 0.0 -0.193397772168782
cpd00053_c L-Glutamine 0.0 -0.261005044219309
cpd00054_c L-Serine 0.0 -0.216753011604418
cpd00060_c L-Methionine 0.0 -0.11349826883634
cpd00065_c L-Tryptophan 0.0 -0.0544955586831525
cpd00066_c L-Phenylalanine 0.0 -0.176188648374102
cpd00069_c L-Tyrosine 0.0 -0.111039822579957
cpd00084_c L-Cysteine 0.0 -0.0569540049395353
cpd00107_c L-Leucine 0.0 -0.347460404235438
cpd00119_c L-Histidine 0.0 -0.0819482085460939
cpd00129_c L-Prol

In [65]:
total = 0
for t in biomass_components:
    total += biomass_components[t]['coefficient']
    print(t, biomass_components[t]['coefficient'])
total

rna 0.0655
dna 0.026
protein 0.5284
energy 41.257
other 0
cofactor 0.1
lipid 0.075
cellwall 0.25


42.3019

In [89]:
for c in biomass_components['cellwall']['components']:
    print(c.compound.id, c.coefficient, c.coefficient_type, c.component_class, biomass_components['cellwall']['coefficient'])
    print(bio1[f'{c.compound.id}0'])

cpd15748_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd15757_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd15766_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd15775_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd15667_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd15668_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd15669_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd11459_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd02229_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783
cpd15665_c -1 MOLSPLIT cellwall 0.25
-0.00309563020839783


In [66]:
c = biomass_components['cellwall']['components'][0]

In [84]:
print(c.coefficient, c.coefficient_type, c.component_class, biomass_components['cellwall']['coefficient'])

-1 MOLSPLIT cellwall 0.25


In [67]:
c.compound

Metabolite identifier,cpd15748_c
Name,"Stearoyllipoteichoic acid (n=24), linked,..."
Memory address,0x07f09dabb6b80
Formula,C123H240O135P24
Compartment,c
In 5 reaction(s),"rxn10318_c, rxn10300_c, rxn10309_c, rxn10200_c, rxn10291_c"


In [78]:
bio1[f'{c.compound.id}0']

-0.00309563020839783

In [75]:
bio1 = dict(map(lambda x: (x[0].id, x[1]), model.reactions.bio1.metabolites.items()))